# Fonctions Utiles 

## Variables Paramètres

In [1]:
KEY = ""

In [2]:
import requests
import pandas as pd
import json
import time

## Mémoïzation ?
Affin d'éviter de demander plusieurs fois la même chose, on pourrait avoir une classe 'Joueur' qui permet de centraliser les infos, pour que toutes les méthodes y aient accès. Dans ce cas, les fonctions de calcul de données deviendraient des méthodes de cette classe.

In [3]:
class Player:
    pass

## Gestion des "bad requests"

In [7]:
def badRequestsHandler(url):
    """
    Permet d'analyser le code de retour de la requête pour gérer les problèmes éventuels.
    Pas sûr que cela permette de gérer tous les problèmes, certains seront probablement spécifiques.
    """
    r = requests.get(url)
    if r.status_code == 429:
        time.sleep(time.sleep(int(r.headers["Retry-After"])+1))
        print("Quota dépassé ...", end = "\r")
        r = requests.get(url)
    elif r.status_code == 400:
        raise Exception("Requête invalide")
    elif r.status_code == 401:
        raise Exception("Unauthorized")
    elif r.status_code == 403:
        raise Exception("Non autorisé: vérifiez la clé")
    elif r.status_code == 404:
        raise Exception("Données non trouvées")
    elif r.status_code == 405:
        raise Exception("Méthode non autorisée")
    elif r.status_code == 415:
        raise Exception("Unsupported media type")
    elif r.status_code == 500:
        raise Exception("Internal server error")
    elif r.status_code == 502:
        raise Exception("Bad gateway")
    elif r.status_code == 503:
        raise Exception("Service non disponible")
    elif r.status_code == 504:
        raise Exception("Gateway timeout")
    elif r.status_code == 200:
        return r.json()

## Récupération des données brutes d'un joueur

In [8]:
def requestSummonerInfo(summoner_name,key):
    """
    Renvoie un dictionnaire contenant des infos sur le joueur 'summoner_name', à savoir :
    accountId :    string - Encrypted account ID. Max length 56 characters.
    profileIconId:    int - ID of the summoner icon associated with the summoner.
    revisionDate:    long - Date summoner was last modified specified as epoch milliseconds. The following events will update this timestamp: summoner name change, summoner level change, or profile icon change.
    name:          string - Summoner name.
    id:            string - Encrypted summoner ID. Max length 63 characters.
    puuid:         string - Encrypted PUUID. Exact length of 78 characters.
    summonerLevel:   long - Summoner level associated with the summoner.
    """
    return badRequestsHandler(f"https://euw1.api.riotgames.com/lol/summoner/v4/summoners/by-name/{summoner_name}?api_key={key}")

requestSummonerInfo('Skeanz',KEY)

{'id': 'oIQG68UUDKSt7bUQlBtSBLqzKCelfv4jSrzooNJ_L4iosa5jrtahspGZUw',
 'accountId': 's4GxVhnvULshQkAN0icIxFhnOBuY3rcm_dolU43KsJLeAY9HN6RbMrg3',
 'puuid': 'TGabv96Z9DC3LHkHNJoqS4j-HJIjLUqqhoyeeJ4FmJwudqjIdh57uvPZbUQJvGd9xtr0ShdUilvzpw',
 'name': 'Skeanz',
 'profileIconId': 7,
 'revisionDate': 1666001193416,
 'summonerLevel': 301}

In [22]:
def requestMostRecentGamesId(puuid,key, nb_of_games=None, nb_of_days_ago=None):
    """
    Si nb_of_games    != None : Récupère les 'nb_of_games' dernières parties jouées par le joueur.
    Si nb_of_days_ago != None : Récupère toutes les parties jouées par le joueur dans les 'nb_of_days_ago' jours.
    Si les deux sont != None ou les deux sont == None, renvoie une erreur.
    Renvoie une liste contenant les id de ces parties.
    """
    if (nb_of_games==None and nb_of_days_ago==None) or (nb_of_games != None and nb_of_days_ago != None):
        raise Exception("Erreur d'inputs, les paramètres nb_of_games et nb_of_days_ago ne peuvent pas tous les deux avoir la même valeur")
    elif nb_of_games != None:
        res_games = []
        start_index = 0 # très important, sinon on récupère en boucle les 100 mêmes games
        while nb_of_games >100:    #la valeur max du "count" pour avoir une liste de match est de 100
            r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start_index}&count=100&api_key={key}")
            start_index+=100
            res_games += r
            nb_of_games-=100

        r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?start={start_index}&count={nb_of_games}&api_key={key}")
        res_games += r

    elif nb_of_days_ago != None :
        res_games = []
        start_index = 0
        startTime = int(time.time()-24*60*60*nb_of_days_ago) # l'horaire de départ est "aujourd'hui"- nb_of_days_ago si on veut les games dans les nb_of_days_ago derniers jours
        while True: 
            r = badRequestsHandler(f"https://europe.api.riotgames.com/lol/match/v5/matches/by-puuid/{puuid}/ids?startTime={startTime}&start={start_index}&count=100&api_key={key}")
            if r == [] : #quand il n'y a plus de games à récup, on sort de la boucle
                break
            start_index+=100
            res_games += r
    return res_games

# GamesId = requestMostRecentGamesId('TGabv96Z9DC3LHkHNJoqS4j-HJIjLUqqhoyeeJ4FmJwudqjIdh57uvPZbUQJvGd9xtr0ShdUilvzpw',KEY,nb_of_games=300)
# GamesId_7_last_days = requestMostRecentGamesId('TGabv96Z9DC3LHkHNJoqS4j-HJIjLUqqhoyeeJ4FmJwudqjIdh57uvPZbUQJvGd9xtr0ShdUilvzpw',KEY,nb_of_days_ago=7)
# print(GamesId_7_last_days)

['EUW1_6125263461', 'EUW1_6125117238', 'EUW1_6124858249', 'EUW1_6124479142', 'EUW1_6124328934', 'EUW1_6124224078', 'EUW1_6123544402', 'EUW1_6123437590', 'EUW1_6123115704', 'EUW1_6122939859', 'EUW1_6122311766', 'EUW1_6122280097', 'EUW1_6122266350', 'EUW1_6122130128', 'EUW1_6122087456', 'EUW1_6122153257', 'EUW1_6121676073', 'EUW1_6121471739', 'EUW1_6121240149', 'EUW1_6121186385', 'EUW1_6121182663', 'EUW1_6121070614', 'EUW1_6121087165', 'EUW1_6120864661', 'EUW1_6120478992', 'EUW1_6120428858', 'EUW1_6120383584', 'EUW1_6119901354', 'EUW1_6119866322', 'EUW1_6119739967', 'EUW1_6119702750', 'EUW1_6119485913', 'EUW1_6119380569', 'EUW1_6118284482', 'EUW1_6118179192', 'EUW1_6118173125', 'EUW1_6118078590', 'EUW1_6118094453', 'EUW1_6117779373', 'EUW1_6117647516', 'EUW1_6116770091', 'EUW1_6116803136', 'EUW1_6116659484', 'EUW1_6116734363', 'EUW1_6116642868', 'EUW1_6116567530', 'EUW1_6116526149', 'EUW1_6116402165', 'EUW1_6116399559', 'EUW1_6115469282']


In [ ]:
def requestRankedInfo(summoner_id):
    """
    Renvoie un dictionnaire de dictionnaire contenant des infos sur les résultats du joueur en partie classées, à savoir :
    FLEXQ: {
        tier: string - Iron -> Challenger
        rank: string - IV -> I
        lp: int - league points
        wins: int
        losses: int
    }
    SOLOQ: {pareil}
    """
    pass

## Calculs de données

In [ ]:
def getWinrate(nb_of_games=0):
    """
    Calcule le winrate (float) et le KDA moyen (float,float,float) sur les 'nb_of_games' dernières parties.
    Si nb_of_games = 0, alors on fait le calcul sur toute la saison.
    Renvoie un dictionnaire de la forme:
    {
        wins: int
        losses: int
        winrate: float
        meanKDA: (float,float,float)
    }
    """
    pass

In [ ]:
def getWinrateOnChampions(nb_of_games=0):
    """
    Calcule le winrate et le KDA moyen sur chaque champion sur les 'nb_of_games' dernières parties
    Si nb_of_games = 0, alors on fait le calcul sur toute la saison.
    Renvoie un dictionnaire de la forme :
    {
        Aatrox: {
            wins: int
            losses: int
            winrate: float
            meanKDA: (float,float,float)
        }
        ...
    }
    """
    pass